In [2]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os

In [3]:
df = pd.read_csv('processed.csv').dropna(how='any')
df.head()

,Unnamed: 0,kepoi_name,koi_disposition,koi_pdisposition,rowid,kepid,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_prad,koi_insol,koi_model_snr
0,0,K00752.01,CONFIRMED,CANDIDATE,1,10797460,0,0,0,0,...,-0.175923,-0.174274,0.149275,-0.411686,-0.385260,0.385260,-0.281810,-0.032699,-0.048067,-0.281610
1,1,K00752.02,CONFIRMED,CANDIDATE,2,10797460,0,0,0,0,...,-0.044527,-0.201766,-0.088371,-0.172241,-0.334348,0.334348,-0.278660,-0.032514,-0.048598,-0.294177
2,2,K00753.01,FALSE POSITIVE,FALSE POSITIVE,3,10811496,0,1,0,0,...,0.069847,0.336068,0.204485,-0.593305,-0.456626,0.456626,-0.157619,-0.028690,-0.048408,-0.230716
3,3,K00754.01,FALSE POSITIVE,FALSE POSITIVE,4,10848459,0,1,0,0,...,0.161526,-0.195821,0.192483,-0.496846,-0.499520,0.499520,-0.191056,-0.022561,-0.043052,0.308766
4,4,K00755.01,CONFIRMED,CANDIDATE,5,10854555,0,0,0,0,...,-0.010185,-0.183084,-0.116376,-0.613039,-0.444831,0.444831,-0.281962,-0.032540,-0.042838,-0.275201


In [4]:
target = df["koi_pdisposition"]
target_names = ['candidate','false positive']

In [5]:
target = target.replace("CANDIDATE",1).replace("FALSE POSITIVE",0)
target.unique()

array([1, 0], dtype=int64)

In [6]:
features = df.drop(['kepoi_name','koi_disposition','koi_pdisposition','rowid','kepid'],axis=1)
feature_names = features.columns
feature_names

Index(['Unnamed: 0', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_score', 'koi_period_err1', 'koi_period_err2',
       'koi_impact', 'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_prad',
       'koi_insol', 'koi_model_snr'],
      dtype='object')

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=419)

In [8]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [9]:
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.989


In [11]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3, n_jobs=-1)

In [ ]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  8.3min


In [1]:
print(grid.best_params_)
print(grid.best_score_)
predictions = grid.predict(X_test)

NameError: name 'grid' is not defined

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

In [ ]:
import joblib
joblib.dump(model, 'model_SVM.sav')